<a href="https://colab.research.google.com/github/cmudrc/sae/blob/master/examples/utility_function_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates three unique ways to compute the utility of a vehicle. The outcomes from each case may or may not align! The three approaches are:
1. Partworth utility over a measure of quality for every unique component of the car
2. Quadratic utility over a measure of quality for every unique component of the car
2. Quadratic utility over the performance attributes of the car, each of which relies on several components

## Install and Import

In [1]:
!pip install git+https://github.com/cmudrc/sae.git

  Cloning https://github.com/cmudrc/sae.git to /tmp/pip-req-build-m8fb5j5i
  Running command git clone --filter=blob:none --quiet https://github.com/cmudrc/sae.git /tmp/pip-req-build-m8fb5j5i
  Resolved https://github.com/cmudrc/sae.git to commit ce6a789e0b810785ba4fb570dcbea5b40848508f
  Preparing metadata (setup.py) ... done
  Created wheel for sae: filename=sae-0.1.0-py3-none-any.whl size=26146 sha256=c875573c5df75f9da76c8c09fd28182801e67aa8f7732e387398a756caff1d82
  Stored in directory: /tmp/pip-ephem-wheel-cache-5rgx4t57/wheels/8d/20/04/8e9bbfdfb118386e3253608654c865b42b674862a738800cfb
Successfully built sae


In [10]:
import numpy
from numpy.typing import NDArray
from typing import Tuple
import sae

## Generate two random cars for us to examine

In [3]:
example_car_1 = sae.Car()
example_car_2 = sae.Car()

# Simulating Random Customers with Partworth Utility

In [4]:
# Get weights for a random customer that define their preferences
weights = numpy.random.rand(10)
print(weights)

[0.80796276 0.22177387 0.8828941  0.38631095 0.90739784 0.5546031
 0.83253125 0.36430607 0.6914742  0.10234462]


In [5]:
# Evaluate the car with respect to those weights. Note that the returned values
# are scaled according to approximate bounds and represent utility*-1 to provide
# an implicit minimization framing
example_car_1.partworth_objectives(weights=weights)

(-1.0028852531019703,
 array([-0.00828908, -0.19115229, -0.00247374, -0.01126424, -0.03145214,
        -0.41163063, -0.3281428 , -0.3718878 , -0.28225456, -0.84603138]))

In [6]:
example_car_2.partworth_objectives(weights=weights)

(-1.234540242718953,
 array([-0.77745905, -0.02698055, -0.1038017 , -0.01007241, -0.00431661,
        -0.14338959, -0.24939046, -0.28994642, -0.10844283, -0.32420369]))

In [7]:
# Assuming the customer acts rationally with respect to this preference model,
# they should buy the following car:
"Car 1" if example_car_1.partworth_objectives(weights=weights)[0] < example_car_2.partworth_objectives(weights=weights)[0] else "Car 2"

'Car 2'

# Simulating Random Customers with Quadratic Utility over Parts

In [ ]:
# The point of simulating customers with more complex utility functions is that
# sometimes more isn't necessarily better! For this

In [23]:
def quadratic_utility_over_parts(car: sae.Car, weights: NDArray) -> Tuple[float, list]:
    partworth = -car.partworth_objectives()[1]
    quadratic_partworth = []
    quadratic_utility = 0.0
    for idx, part in enumerate(list(partworth)):
        quadratic_part = weights[idx,0] + weights[idx,1]*part + weights[idx,2]*part*part
        quadratic_utility -= quadratic_part
        quadratic_partworth.append(-quadratic_part)

    return (quadratic_utility, quadratic_partworth)

In [24]:
# Define the mean form for our weights - this should yield a "leveling out" as
# parts become more performant
mean = numpy.array([[0.0, 2.0, -1.0]] * 10)

In [26]:
# We can now add some variance to each of the weights
weights = mean + 0.1*numpy.random.randn(10, 3)

In [30]:
# With these new weights, we can compute the utility
quadratic_utility_over_parts(example_car_1, weights)

(-4.071488205948333,
 [-0.03126401575296759,
  -0.49089609191384587,
  0.10999728521578471,
  -0.1251560690047339,
  -0.23026173716869616,
  -0.8025663995233542,
  -0.6015243863484405,
  -0.6218019671511774,
  -0.4599948061092836,
  -0.8180200181916184])

In [34]:
quadratic_utility_over_parts(example_car_2, weights)

(-3.5374681841143047,
 [-0.9674580251962474,
  -0.17123561612296972,
  -0.091656557801661,
  -0.1227515219969897,
  -0.1722880595563746,
  -0.3619720050640406,
  -0.47527137316762497,
  -0.5237436374864741,
  -0.17106735660811145,
  -0.4800240311138118])

In [32]:
# Assuming the customer acts rationally with respect to this preference model,
# they should buy the following car:
"Car 1" if quadratic_utility_over_parts(example_car_1, weights)[0] < quadratic_utility_over_parts(example_car_2, weights)[0] else "Car 2"

'Car 1'

# Simulating Random Customers with Quadratic Utility over Performance

In [ ]:
# The point of simulating customers with more complex utility functions is that
# sometimes more isn't necessarily better!

In [42]:
def quadratic_utility_over_performance(car: sae.Car, weights: NDArray) -> Tuple[float, list]:
    partworth = car.objectives()[1]
    quadratic_partworth = []
    quadratic_utility = 0.0
    for idx, part in enumerate(list(partworth)):
        quadratic_part = weights[idx,0] + weights[idx,1]*part + weights[idx,2]*part*part
        quadratic_utility -= quadratic_part
        quadratic_partworth.append(-quadratic_part)

    return (quadratic_utility, quadratic_partworth)

In [43]:
# Define the mean form for our weights - this should yield a "leveling out" as
# parts become more performant
mean = numpy.array([[0.0, 2.0, -1.0]] * 11)

In [44]:
# We can now add some variance to each of the weights
weights = mean + 0.1*numpy.random.randn(11, 3)

In [45]:
# With these new weights, we can compute the utility
quadratic_utility_over_performance(example_car_1, weights)

(-4.903078439305462,
 [0.0981353783732669,
  -0.2994749708779486,
  -0.20648371506368626,
  -1.0953602066268822,
  -1.0351736151108994,
  -0.19673768920421758,
  -0.09611383594573847,
  -1.25267105164887,
  -0.14563029029530491,
  -0.1956765490085978,
  -0.4778918938965845])

In [46]:
quadratic_utility_over_performance(example_car_2, weights)

(-6.705037303255647,
 [-0.0224891939112073,
  -0.8982419122961699,
  -0.7106703330066542,
  -0.7165058036911959,
  -1.034878063603879,
  -0.3989786308510101,
  -0.3858572049689103,
  -0.9292412429505923,
  -0.30438664934800563,
  -0.5783186742202403,
  -0.7254695944077834])

In [47]:
# Assuming the customer acts rationally with respect to this preference model,
# they should buy the following car:
"Car 1" if quadratic_utility_over_parts(example_car_1, weights)[0] < quadratic_utility_over_parts(example_car_2, weights)[0] else "Car 2"

'Car 1'